<a href="https://colab.research.google.com/github/MichaelMcCarey/SteelProject/blob/main/SteelProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import useful libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# import dataset
steel = 'https://raw.githubusercontent.com/MichaelMcCarey/SteelProject/main/steel.csv'
steel_data = pd.read_csv(steel)

In [3]:
# check to make sure data was imported correctly
print(steel_data.head())
print(steel_data.shape)

   normalising_temperature  tempering_temperature  percent_silicon  \
0                  178.500                    275           0.1530   
1                  178.500                    950           0.1530   
2                  178.500                    375           0.1530   
3                  178.500                    900           0.1530   
4                  189.525                    900           0.1624   

   percent_chromium  percent_copper  percent_nickel  percent_sulphur  \
0          0.970575           0.942          0.8870              0.0   
1          1.212726           0.942          0.8870              0.0   
2          1.621165           0.942          0.8870              0.0   
3          0.809989           0.942          0.8870              0.0   
4          1.036229           0.849          0.9382              0.0   

   percent_carbon  percent_manganese  tensile_strength  
0           1.920                0.0         25.107613  
1           1.920               

In [10]:
# import k-fold cross validation and the two algorithims
from sklearn.model_selection import KFold, GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [5]:
# seperate the independent and dependent variables
dependent_cols = ['tensile_strength']
independent_cols = ['normalising_temperature', 'tempering_temperature', 'percent_silicon', 'percent_chromium', 'percent_copper', 'percent_nickel', 'percent_sulphur', 'percent_carbon', 'percent_manganese']

In [6]:
# set up matrix
x = steel_data[independent_cols]
y = steel_data[dependent_cols]
print(x.shape)
print(y.shape)

(553, 9)
(553, 1)


In [64]:
# setting up for 10 fold cross validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

In [65]:
# using r2 as the domain independent measure of error and RMSE as the domain specific measure of error
scoring = 'r2', 'neg_root_mean_squared_error'

In [66]:
# KNN model with default hyperparameters
# need to use pipeline with standard scaling because knn is much more susceptible to the effects of relative scale
knn_pipeline = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsRegressor())])

In [67]:
# cross validate using kfold that was set up previously
knn_scores = cross_validate(knn_pipeline, x, y, cv=kfold, scoring=scoring, return_train_score=True)

In [68]:
# get the average scores for each measure of error for training and testing
knn_train_r2_score = knn_scores['train_r2'].mean()
knn_test_r2_score = knn_scores['test_r2'].mean()
# rmse will return a negative number so need to change the sign with a negative
knn_train_rmse_score = -knn_scores['train_neg_root_mean_squared_error'].mean()
knn_test_rmse_score = -knn_scores['test_neg_root_mean_squared_error'].mean()

In [69]:
print(knn_train_r2_score)
print(knn_test_r2_score)
print(knn_train_rmse_score)
print(knn_test_rmse_score)

0.857842272435416
0.7658132198063687
34.3076604870604
42.65590640074778


In [70]:
# set up param grid for gridsearch
knn_param_grid = {'knn__n_neighbors': range(1,20), 'knn__weights': ['uniform', 'distance']} # need to use __ when using pipeline to fix error message

In [71]:
# perform gridsearch
knn_gridsearch = GridSearchCV(estimator=knn_pipeline, param_grid=knn_param_grid, cv=kfold, scoring=scoring, refit = 'r2', return_train_score=True)

In [72]:
knn_gridsearch.fit(x, y)

GridSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('knn', KNeighborsRegressor())]),
             param_grid={'knn__n_neighbors': range(1, 20),
                         'knn__weights': ['uniform', 'distance']},
             refit='r2', return_train_score=True,
             scoring=('r2', 'neg_root_mean_squared_error'))

In [73]:
knn_gridsearch.best_score_

np.float64(0.8030817784218238)

In [32]:
knn_best = knn_gridsearch.best_estimator_
knn_best

Pipeline(steps=[('scaler', StandardScaler()),
                ('knn',
                 KNeighborsRegressor(n_neighbors=7, weights='distance'))])

In [33]:
# Decision tree model with default hyperparameters
model = DecisionTreeRegressor()

In [34]:
# cross validate using kfold that was set up previously
tree_scores = cross_validate(model, x, y, cv=kfold, scoring=scoring, return_train_score=True)

In [35]:
# get the average scores for each measure of error for training and testing
tree_train_r2_score = tree_scores['train_r2'].mean()
tree_test_r2_score = tree_scores['test_r2'].mean()
# rmse will return a negative number so need to change the sign with a negative
tree_train_rmse_score = -tree_scores['train_neg_root_mean_squared_error'].mean()
tree_test_rmse_score = -tree_scores['test_neg_root_mean_squared_error'].mean()

In [36]:
print(tree_train_r2_score)
print(tree_test_r2_score)
print(tree_train_rmse_score)
print(tree_test_rmse_score)

1.0
0.806131487411131
1.7656492384183352e-15
38.37000413754504


In [89]:
# set up param grid
tree_param_grid = {'max_depth': list(range(1,16)) + [None], 'min_samples_leaf': range(1,11)} # have to use list to include integer range and None

In [90]:
# perform gridsearch
tree_gridsearch = GridSearchCV(estimator=model, param_grid=tree_param_grid, cv=kfold, scoring=scoring, refit = 'r2', return_train_score=True)

In [91]:

tree_gridsearch.fit(x, y)

GridSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
             estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, None],
                         'min_samples_leaf': range(1, 11)},
             refit='r2', return_train_score=True,
             scoring=('r2', 'neg_root_mean_squared_error'))

In [92]:
tree_gridsearch.best_score_

np.float64(0.8246083868062684)